In [1]:
!pip install -q langchain cohere bs4 faiss-cpu

In [2]:
!pip install -q typing-inspect==0.8.0 typing_extensions==4.5.0 pydantic==1.10.11

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatCohere
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS

In [4]:
llm = ChatCohere(cohere_api_key="nMfqYUZJoPhJ3uI6aJq573MEHa9MsXxtSUGRVw5f")
embeddings = CohereEmbeddings(cohere_api_key="nMfqYUZJoPhJ3uI6aJq573MEHa9MsXxtSUGRVw5f")

In [5]:
loader = WebBaseLoader("https://jalammar.github.io/illustrated-transformer/")
docs = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
vector_store = FAISS.from_documents(documents, embeddings)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "What is a transformer?"})
response

{'input': 'What is a transformer?',
 'context': [Document(page_content='Go Forth And Transform\nI hope you’ve found this a useful place to start to break the ice with the major concepts of the Transformer. If you want to go deeper, I’d suggest these next steps:', metadata={'source': 'https://jalammar.github.io/illustrated-transformer/', 'title': 'The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time.', 'description': 'Discussions:\nHacker News (65 points, 4 comments), Reddit r/MachineLearning (29 points, 3 comments)\n\n\nTranslations: Arabic, Chinese (Simplified) 1, Chinese (Simplified) 2, French 1, French 2, Italian, Japanese, Korean, Persian, Russian, Spanish 1, Spanish 2, Vietnamese\n\nWatch: MIT’s Deep Learning State of the Art lecture referencing this post\n\nFeatured in courses at Stanford, Harvard, MIT, Princeton, CMU and others\n\nIn the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention

In [16]:
response["answer"]

'Transformers are a type of machine-learning model behind NLP tasks. They rely entirely on attention mechanisms to handle long-range dependencies and context by attending to different parts of the input sequence. This makes them ideal for tasks involving sequential or time-series data like NLP tasks. They are called "transformers" because they use the same weight matrix to transform different elements of the input sequence.\n\nWould you like me to go into more detail about transformers?'